# Point mutations in NTRKs

In [ ]:
from openeye import oechem
import MDAnalysis as mda

input_pdb_path = "../md/data/md_equil/4YNE/output/"

ligand_list = ["larotrectinib", "selitrectinib", "repotrectinib"]

ligand_input_pdb_dict = {
    "lar" : input_pdb_path + "larotrectinib/equilibrated_state_5ns.pdb",
    "sel" : input_pdb_path + "selitrectinib/equilibrated_state_5ns.pdb",
    "rep" : input_pdb_path + "repotrectinib/equilibrated_state_5ns.pdb"
}

# Strip out protein and ligand coordinates

for ligand_resname in ligand_input_pdb_dict:

    u = mda.Universe(ligand_input_pdb_dict[ligand_resname])

    protein = u.select_atoms("protein or resname ACE NME")
    protein.write("./data/4yne_" + ligand_resname + "_apo.pdb")

    ligand = u.select_atoms("resname " + ligand_resname)
    ligand.write("./data/" + ligand_resname + ".pdb")

# Create SDF files for each ligand to be used later

for ligand_resname in ligand_input_pdb_dict:

    ifs = oechem.oemolistream("./data/" + ligand_resname + ".pdb")
    ofs = oechem.oemolostream("./data/" + ligand_resname + ".sdf")

    ifs.SetFormat(oechem.OEFormat_PDB)
    ofs.SetFormat(oechem.OEFormat_SDF)

    for mol in ifs.GetOEGraphMols():
        oechem.OEWriteMolecule(ofs, mol)

    

In [ ]:
# Set up logger
import logging
_logger = logging.getLogger()
_logger.setLevel(logging.DEBUG)

In [ ]:
# Solvent
from perses.app.relative_point_mutation_setup import PointMutationExecutor
from simtk import unit
solvent_delivery = PointMutationExecutor("./data/4yne_lar_apo.pdb", 
                        '1', # First and only protein chain 
                        '667', 
                        'CYS',
                        ligand_file="./data/lar.sdf",
                        ionic_strength=0.15*unit.molar
                       )